# 1. Initializations

## 1.1 General imports

In [ ]:
### Data management
import pandas as pd
import numpy as np
import random

### Machine Learning

# transformation
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# resampling
from imblearn.over_sampling import SMOTE

# metrics and evaluation
from sklearn.metrics import f1_score, confusion_matrix
from scipy.stats import chi2_contingency, probplot
from xgboost import plot_importance

### Data Viz

# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline

# graphical seaborn
import seaborn as sns

# # graphical plotly
# import plotly.graph_objects as go
# import plotly.express as px
# # for jupyter notebook display management
# import plotly.io as pio
# pio.renderers.default = "notebook"


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General Classification functions

In [ ]:
import smartcheck.classification_common as cls

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

### 2.1.1 VELIB DISPO (Optional Dataset)

#### Loading and column management (columns names normalization)

In [ ]:
df_disp_velib_raw = dfc.load_dataset_from_config('velib_dispo_data', sep=';')

if df_disp_velib_raw is not None and isinstance(df_disp_velib_raw, pd.DataFrame):
    display(df_disp_velib_raw.head())
    dfc.log_general_info(df_disp_velib_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_disp_velib_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_disp_velib_raw))
    df_disp_velib = dfc.normalize_column_names(df_disp_velib_raw)

#### Global description and correlation

In [ ]:
df_disp_velib.info()
display(df_disp_velib.head())
df_cpt_velo_desc = df_disp_velib.select_dtypes(include=np.number).describe()
display(df_cpt_velo_desc)
df_cpt_velo_cr = df_disp_velib.select_dtypes(include=np.number).corr()
display(df_cpt_velo_cr)
dfc.display_variable_info(df_disp_velib, 5)

#### Cross Distribution inspection

In [ ]:
# Analyse de la distribution d'une variable spécique en relation avec les autres de son dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
dfc.analyze_by_reference_variable(df_disp_velib, ref_col)

In [ ]:
# Analyse croisée de la distribution d'une variable spécifique en fonction d'autres variables (quantitatives ou qualitatives) du dataframe
ref_col = 'station_en_fonctionnement' # ici notre variable cible
cross_columns = [ref_col] + ['borne_de_paiement_disponible', 'retour_velib_possible']
dfc.log_cross_distributions(
    df_disp_velib[cross_columns], 
    ref_col
)

In [ ]:
# Analyse croisée d'une variable en fonction d'une autre
ref_col = 'borne_de_paiement_disponible'
target_col = 'station_en_fonctionnement'
display(pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col]))

In [ ]:
# Analyse croisée des d'une variable en fonction d'une autre (en conservant les NaN)
ref_col = 'station_opening_hours'
target_col = 'station_en_fonctionnement'
ref_cross_tab = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab)

# Catégorisation et normalisation
ref_col_val_norm = np.where(
    df_disp_velib[ref_col].isin(ref_cross_tab[ref_cross_tab['OUI'] >= 0.8].index.tolist()), 
    1, 
    0
)
df_disp_velib[ref_col] = ref_col_val_norm
ref_cross_tab_norm = pd.crosstab(df_disp_velib[ref_col], df_disp_velib[target_col], dropna=False, normalize=True)
display(ref_cross_tab_norm)

#### Signifiance against target evaluation

In [ ]:
# Verification de la signifiance des variables explicatives par rapport à une variable cible
ref_col = 'retour_velib_possible'
target_col = 'station_en_fonctionnement'
# Génération des colonnes dummies pour ref_col
ref_col_dummies = pd.get_dummies(df_disp_velib[ref_col], prefix=ref_col)
print("Colonnes dummies générées :", list(ref_col_dummies.columns))
# Pour chaque modalité de cette variable (dummy 0/1), tester sa signifiance avec la variable cible
for col in ref_col_dummies:
    # Test du Chi-Deux
    cross_tab = pd.crosstab(df_disp_velib[target_col], ref_col_dummies[col])
    if cross_tab.shape[1] != 2:
        print(f"⚠️ Modalité [{col}] ignorée (1 seule valeur présente)")
        continue
    stat, p, _, _ = chi2_contingency(cross_tab)
    # V de Cramer
    V_Cramer = np.sqrt(
        stat/cross_tab.values.sum())
    # On affiche uniquement les variables significatives et dont le V de Cramer est supérieur à 0.1
    # Faible : Valeur autour de 0.1 ;
    # Moyenne : Valeur autour de 0.3 ;
    # Elevée : Valeur autour et supérieure à 0.5.
    # Lorsque la valeur du V de Cramer est très élevée (aux alentours de 0.8 et plus), on soupçonne généralement de la multicolinéarité.
    result = 'significative' if (p < 0.05) and (V_Cramer > 0.1) else 'NON signficative'
    print(f"Variable [{col}] {result} Vs [{target_col}]: p-value[{p:.5f}], V_Cramer[{V_Cramer:.5f}]")

### 2.1.2 VELO COMPTAGE (Main Data Set)

#### Loading and column management (columns names normalization)

In [ ]:
df_cpt_velo_raw = dfc.load_dataset_from_config('velo_comptage_data', sep=';')

if df_cpt_velo_raw is not None and isinstance(df_cpt_velo_raw, pd.DataFrame):
    display(df_cpt_velo_raw.head())
    dfc.log_general_info(df_cpt_velo_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_cpt_velo_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_cpt_velo_raw))
    df_cpt_velo = dfc.normalize_column_names(df_cpt_velo_raw)

#### Global description and correlation

In [ ]:
df_cpt_velo.info()
display(df_cpt_velo.head())
df_cpt_velo_desc = df_cpt_velo.select_dtypes(include=np.number).describe()
display(df_cpt_velo_desc)
df_cpt_velo_cr = df_cpt_velo.select_dtypes(include=np.number).corr()
display(df_cpt_velo_cr)

## 2.2 Data quality refinement

### 2.2.2 VELIB DISPO (Optional Dataset)

In [ ]:
# Original backup and duplicates management
df_disp_velib_orig = df_disp_velib.copy()
df_disp_velib = df_disp_velib.drop_duplicates()

In [ ]:
df_disp_velib.station_en_fonctionnement = df_disp_velib.station_en_fonctionnement.apply(lambda x: 1 if x=='OUI' else 0).astype(int)

In [ ]:
# Exemple de modification localisée en fonction de la proximité à la médiane d'autre variables
# mask = (
#     (train['Gender'].isna()) &
#     (abs(train['Age'] - 30) > abs(train['Age'] - 41)) & # L’âge est plus proche de 41 que de 30
#     (train['Previously_Insured'] == 0) & # La personne n’était pas assurée auparavant
#     (train['Vehicle_Damage'] == 1) # Elle a subi un dommage sur son véhicule
# )
# train.loc[mask, 'Gender'] = 0

In [ ]:
# Exemple de modification par répartition spécifique entre deux valeurs 0 et 1
# proportion_tab = [0] * 55 + [1] * 45
# mask = (
#     (train['Gender'].isna()) &
# )
# train.loc[mask, 'Gender'] = train.loc[mask, 'Gender'].apply(lambda x: random.choice(proportion))

In [ ]:
# Preprocessing par scaling des données 
# - ni outlier ni distribution loi normale : min/max
# - sans outlier mais distribution loi normale : standard
# - avec outlier : Robust 
mm_scaler = MinMaxScaler()
r_scaler = RobustScaler()
s_scaler = StandardScaler()

r_scale_col = ['nombre_bornettes_libres', 'nombre_total_velos_disponibles']
df_disp_velib[r_scale_col] = s_scaler.fit_transform(df_disp_velib[r_scale_col])

### 2.2.2 VELO COMPTAGE (Main Dataset)

In [ ]:
# Original backup and duplicates management
df_cpt_velib_orig = df_cpt_velo.copy()
df_cpt_velo = df_cpt_velo.drop_duplicates()

## 2.3 Feature engineering and combination

#### Feature refactoring

#### Optional and main data set combination/merge

# 2. Data Viz' and Analysis

## 2.1 General Data Viz'

In [ ]:
# Vérificationn graphique de la répartition en loi normale de chaque données numérique
for col in df_disp_velib.select_dtypes(include='number').columns:
    probplot(df_disp_velib[col], dist="norm", plot=plt)
    plt.suptitle(f"Column {col}")
    plt.show()

## 2.2 Quantitative mono variable distribution

## 2.3 Qualitative mono variable distribution

## 2.4 Qualitative multi variable distribution

## 2.5 Quantitative multi variable correlation

# 3. Initial machine learning exploration

In [ ]:
# Separation base et valid
df_base, df_valid = train_test_split(df_disp_velib, test_size=0.1, random_state=66)
df_base_X = df_base.drop('station_en_fonctionnement', axis=1)
# ajustement : enlever les variables non retravaillées pour le moment
df_base_X = df_base_X.drop(columns=df_base_X.select_dtypes(exclude='number').columns)
df_base_X = df_base_X.drop(columns=['code_insee_communes_equipees', 
                                    'station_opening_hours', 
                                    'velos_mecaniques_disponibles', 
                                    'velos_electriques_disponibles',
                                    'capacite_de_la_station'])
df_base_y = df_base.station_en_fonctionnement
display(df_base_X)
display(df_base_y)

In [ ]:
# Separation base : Train et Test
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(df_base_X, df_base_y, train_size=0.8, random_state=66)
print("Train Set:", X_train_b.shape)
print("Test Set:", X_test_b.shape)

### 3.1 Quick Logistic Regression

In [ ]:
# Logistic Regression configuration
logit_config = {
    "target": "station_en_fonctionnement",
    "features": ['nombre_bornettes_libres', 'nombre_total_velos_disponibles']
}

# Coefficient adjustment configuration
adjustment_config = {
    "nombre_bornettes_libres": {
        "type": "normalize",
        "range": (0, 68)
    },
    "nombre_total_velos_disponibles": {
        "type": "normalize",
        "range": (0, 65)
    },
    # "nombre_total_velos_disponibles": {
    #     "type": "inverse"
    # },
}

subset = logit_config["features"]+[logit_config["target"]]
cls.logit_analysis(df_base[subset], logit_config, adjustment_config)

### 3.2 SMOTE/Under/Over Sampling (if necessary)

In [ ]:
cls.cross_validation_with_resampling(X_train_b, y_train_b, LogisticRegression())

In [ ]:
cls.cross_validation_with_resampling(X_train_b, y_train_b, XGBClassifier(eval_metric="error"))

### 3.3 Adding threashold (if necessary)

In [ ]:
cls.cross_validation_with_resampling_and_threshold(X_train_b, y_train_b, LogisticRegression())

### 3.4 Switching to other models (if necessary)

In [ ]:
cls.cross_validation_with_resampling_and_threshold(X_train_b, y_train_b, XGBClassifier(eval_metric="error"))

### 3.5 Applying Best model and visualizing results

In [ ]:
# Resampling sur la base d'entraînement
smote = SMOTE()
X_train_b_smote, y_train_b_smote = smote.fit_resample(X_train_b, y_train_b)

# Définition et entraînement du modèle
clf_XGB_opti = XGBClassifier(eval_metric="error")
clf_XGB_opti.fit(X_train_b_smote, y_train_b_smote)

# Prédiction du modèle (Seuil précédemment établi)
preds = np.where(clf_XGB_opti.predict_proba(X_test_b)[:, 1] > 0.026, 1, 0)

# Evaluation
print("Modèle avec rééchantillonnage et optimisation du seuil")
print("F1-Score : ", f1_score(preds, y_test_b))
print(confusion_matrix(preds, y_test_b), end="\n\n")

# Modèle de base
print("Modèle de base", end="\n\n")
clf_XGB_base = XGBClassifier(eval_metric="error")
clf_XGB_base.fit(X_train_b, y_train_b)
print("F1-Score : ", f1_score(clf_XGB_base.predict(X_test_b), y_test_b))
print(confusion_matrix(clf_XGB_base.predict(X_test_b), y_test_b))
plot_importance(clf_XGB_base) 
plt.show()